<a href="https://colab.research.google.com/github/elichen/aoc2023/blob/main/Day_19_Aplenty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
input = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

In [ ]:
input = open("input.txt").read().rstrip()

In [ ]:
def parse_data_lines(lines):
    data_list = []
    for line in lines:
        # Remove curly braces and split by comma
        pairs = line.strip('{}').split(',')
        data_dict = {}
        for pair in pairs:
            key, value = pair.split('=')
            data_dict[key] = int(value)  # Convert value to integer
        data_list.append(data_dict)
    return data_list

In [ ]:
# Parse the workflow lines into a dictionary
def parse_workflows(lines):
    workflows = {}
    for line in lines:
        name, conditions_str = line.split('{')
        conditions = conditions_str[:-1].split(',')  # Remove the closing '}' and split by ','
        parsed_conditions = []

        for condition in conditions:
            if ':' in condition:
                condition, target = condition.split(':')
                parsed_conditions.append((condition, target))
            else:
                # Handle the last condition
                parsed_conditions.append(("default", condition))

        workflows[name] = parsed_conditions

    return workflows

In [ ]:
workflows, datas = input.split("\n\n")
datas = parse_data_lines(datas.split('\n'))
workflows = parse_workflows(workflows.split('\n'))

In [ ]:
def traverse_workflows(workflows, start_workflow):
    def traverse(workflow_name, index, path):
        # Termination conditions
        if workflow_name == 'A':
            return [path]
        if workflow_name == 'R':
            return []

        # Access the current workflow and condition
        workflow = workflows.get(workflow_name, [])
        if index < len(workflow):
            condition, target = workflow[index]
        else:
            return []

        # If it's the default condition, take the target unconditionally
        if condition == "default":
            return traverse(target, 0, path)

        # Record the pass and fail paths
        pass_path = traverse(target, 0, path + [condition])
        negated_condition = negate_condition(condition)
        fail_path = traverse(workflow_name, index + 1, path + [negated_condition])

        return pass_path + fail_path

    # Function to negate a condition
    def negate_condition(condition):
        if '<' in condition:
            return condition.replace('<', '>=')
        elif '>' in condition:
            return condition.replace('>', '<=')

    # Start traversal from the specified workflow
    return traverse(start_workflow, 0, [])

# Test the function
result_paths = traverse_workflows(workflows, 'in')

In [ ]:
def calculate_combinations(paths):
    max_value = 4000  # Maximum value for each rating

    # Function to extract key and value from a condition
    def extract_key_value(condition):
        for op in ['<=', '>=', '<', '>']:
            if op in condition:
                key, value = condition.split(op)
                return key, op, int(value)
        return '', '', 0

    # Function to calculate the range of values satisfying a condition
    def calculate_range(key, operator, value):
        if operator == '<':
            return range(1, value)
        elif operator == '<=':
            return range(1, value + 1)
        elif operator == '>':
            return range(value + 1, max_value + 1)
        elif operator == '>=':
            return range(value, max_value + 1)

    # Function to calculate combinations for a single path
    def calculate_path_combinations(path):
        ranges = {'x': range(1, max_value + 1), 'm': range(1, max_value + 1),
                  'a': range(1, max_value + 1), 's': range(1, max_value + 1)}

        for condition in path:
            if condition != 'A':  # Skip 'A' as it's not a condition
                key, operator, value = extract_key_value(condition)
                ranges[key] = set(ranges[key]).intersection(calculate_range(key, operator, value))

        return len(ranges['x']) * len(ranges['m']) * len(ranges['a']) * len(ranges['s'])

    # Calculate combinations for all paths and sum them up
    total_combinations = sum(calculate_path_combinations(path) for path in paths)
    return total_combinations

calculate_combinations(result_paths)

134343280273968